In [ ]:
# | hide
import nbdev
from nbdev.showdoc import *

nbdev.nbdev_export()

### Functional operations


In [ ]:
# | default_exp functional

In [ ]:
# | export
from tidygrad.tensor import Tensor, BaseOp, UnaryElementwiseOp, BinaryElementwiseOp
import numpy as np

In [ ]:
# | exporti
class Sigmoid(UnaryElementwiseOp):
    """Take the sigmoid of a tensor"""

    name_template = "sigmoid({})"

    def __init__(self, a, name=None):
        super().__init__(a, name=name)
        self.out = Tensor(1 / (1 + np.exp(-self.args[0].data)), name=self.name, op=self)

    def backward(self):
        with np.errstate(under="ignore"):  # Triggered by infinitesimally small 1-data
            self.parents[0].grad += self.out.grad * self.out.data * (1 - self.out.data)

In [ ]:
# | export
def sigmoid(input, name=None):
    return Sigmoid(input, name=name).out

In [ ]:
# | exporti
class Relu(UnaryElementwiseOp):
    """Take the sigmoid of a tensor"""

    name_template = "relu({})"

    def __init__(self, a, name=None):
        super().__init__(a, name=name)
        self.out = Tensor(np.maximum(0, self.args[0].data), name=self.name, op=self)

    def backward(self):
        self.parents[0].grad += self.out.grad * (self.out.data > 0)

In [ ]:
# | export
def relu(input, name=None):
    return Relu(input, name=name).out

In [ ]:
# | export
from tidygrad.tensor import ExpLog

In [ ]:
# | export


def BCE_loss(logits, target):
    return -logits + logits * target - ExpLog(-logits).out

In [ ]:
# | exporti
class Dropout(UnaryElementwiseOp):
    """Apply Dropout to a tensor"""

    name_template = "dropout({})"

    def __init__(self, a, p_drop=0.1, training=True, name=None):
        super().__init__(a, name=name)
        assert 0 < p_drop < 1, f"p_drop must in (0, 1), got {p_drop}"
        self.p_drop = p_drop
        self.training = training
        if training:
            # Note: We scale up the outputs during training rather than scaling down during inference.
            scale_factor = 1 / (1 - p_drop)
            self.mask = np.random.binomial(
                scale_factor, 1 - p_drop, size=self.args[0].data.shape
            )
            self.out = Tensor(self.args[0].data * self.mask, name=self.name, op=self)
        else:
            self.out = Tensor(self.args[0].data, name=self.name, op=self)

    def backward(self):
        self.parents[0].grad += self.out.grad * (self.mask if self.training else 1)

In [ ]:
# | export


def dropout(x, p=0.5, training=True):
    if p == 0:
        return x

    return Dropout(x, p_drop=p, training=training).out

In [ ]:
import torch

a = torch.randn(2, 30, requires_grad=True)

b = torch.nn.functional.dropout(a, p=0.1, training=True)

b.sum().backward()

In [ ]:
a.grad

tensor([[1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111,
         1.1111, 1.1111, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111, 1.1111, 1.1111,
         1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111,
         1.1111, 0.0000, 1.1111],
        [1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 0.0000, 1.1111, 0.0000, 1.1111,
         0.0000, 1.1111, 0.0000, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111,
         1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111, 1.1111,
         1.1111, 1.1111, 1.1111]])